In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as pgo
import plotly.subplots as ps

from pathlib import Path

pd.set_option('display.max_rows', 2)

In [3]:
d_root = Path("/home/fpavlov/projects/article_conserved_miRNA")
d_data = d_root / "data"
d_img = d_root / "img"

# Input
f_flipon_to_data = d_data / "flipon_to_data.tsv"
f_flipon_to_data_og = d_data / "flipon_to_data_og.tsv"


In [53]:
def get_group(x: list):
    any_ccre = x[0].replace('(CTCF-bound)', '').replace('CTCF-only', '').replace('CTCF', '').strip(", ") != ''
    any_ctcf = "CTCF" in x[0]
    any_reps = x[1] != ""

    return " & ".join([any_ccre * "cCRE", any_reps * "LINE/LTR", any_ctcf * "CTCF"]).replace(
        "&  &", "&"
    )

def get_plotly_data(path: Path) -> pd.DataFrame:
    flipons_to_data = pd.read_table(path).fillna('')
    result = (
        flipons_to_data.replace(regex=r"(@(.+?), )|(@(.+?)$)", value=",")
        .assign(
            feature_group=lambda x: x[
                ["cCRE (+-200bp slop)", "LINE/LTR repeats (+-200bp slop)"]
            ]
            .apply(get_group, axis=1)
            .str.strip("& ").str.replace('& ', '&<br>')
        )
        .loc[:, ["feature_group", "annotation", "group"]]
        .groupby(["annotation", "feature_group"])[["group"]]
        .value_counts()
        .to_frame()
        .rename(columns={0: "count"})
        .reset_index()
    )
    result = result[result['feature_group'] != ''].replace(
        {"quadruplex": "G4", "z-dna": "Z-DNA", "h-dna": "H-DNA", "sidd": "SIDD"}
    )
    return result


In [54]:
plotly_data = get_plotly_data(f_flipon_to_data)
plotly_data

,annotation,feature_group,group,count
4,3' UTR,CTCF,SIDD,2
...,...,...,...,...
191,Promoter (<=1kb),cCRE &<br>LINE/LTR &<br>CTCF,H-DNA,3


In [55]:
plotly_data_og = get_plotly_data(f_flipon_to_data_og)
plotly_data_og

/tmp/ipykernel_4103817/3702175196.py:11: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



,annotation,feature_group,group,count
4,3' UTR,CTCF,SIDD,2
...,...,...,...,...
218,Promoter (<=1kb),cCRE &<br>LINE/LTR &<br>CTCF,SIDD,11


In [74]:
groups = ["G4", "Z-DNA", "SIDD", "H-DNA"][::-1]
annotations = [
    "Promoter (<=1kb)",
    "Promoter (1-2kb)",
    "Promoter (2-3kb)",
    "5' UTR",
    "3' UTR",
    "Exon",
    "Intron",
    "Downstream (<=300bp)",
    "Distal Intergenic",
]
feature_groups = [
    "cCRE",
    "LINE/LTR",
    "cCRE &<br>LINE/LTR",
    "CTCF",
    "cCRE &<br>CTCF",
    "LINE/LTR &<br>CTCF",
    "cCRE &<br>LINE/LTR &<br>CTCF",
]

n_rows = len(feature_groups)
n_cols = len(annotations)

fig = ps.make_subplots(
    rows=n_rows,
    cols=n_cols,
    shared_xaxes="all",
    shared_yaxes="all",
    horizontal_spacing=0.004,
    vertical_spacing=0.01,
    column_titles=annotations,
    row_titles=feature_groups,
    x_title="Flipon count",
    # y_title="Flipon group",
)

for i, feature_group in enumerate(feature_groups):
    for j, annotation in enumerate(annotations):
        df = (
            plotly_data[
                (plotly_data["feature_group"] == feature_group)
                & (plotly_data["annotation"] == annotation)
            ]
            .set_index("group")
            .reindex(groups)
            .dropna()
        )

        if not len(df):
            continue

        colors = {
            "G4": "#FE9B54",
            "Z-DNA": "royalblue",
            "SIDD": "#ED4241",
            "H-DNA": "darkorchid",
        }
        # https://stackoverflow.com/questions/70347594/how-to-format-plotly-legend-when-using-marker-color
        for idx, row in df.iterrows():
            sub = pgo.Bar(
                x=[row["count"]],
                y=[idx],
                name=idx,
                textposition="outside" if row["count"] < 2100 else "inside",
                texttemplate="%{x:,d}",
                marker=dict(color=colors[idx]),
                width=1,
                orientation="h",
                showlegend=True if (i == 0 and j == 0) else False,
            )
            fig.add_trace(sub, row=i + 1, col=j + 1)
        fig.update_xaxes(ticks="", row=i + 1, col=j + 1)

fig.update_xaxes(tickformat="s", showgrid=False)
fig.update_xaxes(ticks="outside", row=n_rows)
fig.update_yaxes(type="category", visible=False)
fig.update_annotations(textangle=0, font=dict(size=14), align='left')
fig.update_layout(
    title=f"Flipons ({plotly_data['count'].sum():,d})",
    title_x=0.005, title_y=.99,
    height=900,
    width=1600,
    font=dict(family="monospace", size=12),
    uniformtext_minsize=12,
    margin=dict(l=5, t=55, b=55, r=60),
    legend=dict(
        # title_text="Flipon group",
        traceorder="reversed",
        orientation="h",
        yanchor="bottom",
        y=-.07,
        xanchor="right",
        x=0.98,
    ),
    template="ggplot2",
)

fig.write_image(d_img / 'enrichment-table-all-flipons-01-mirna-only.png', scale=2)

fig.show()


In [72]:
n_rows = len(feature_groups)
n_cols = len(annotations)

fig = ps.make_subplots(
    rows=n_rows,
    cols=n_cols,
    shared_xaxes="all",
    shared_yaxes="all",
    horizontal_spacing=0.004,
    vertical_spacing=0.01,
    column_titles=annotations,
    row_titles=feature_groups,
    x_title="Flipon count",
    # y_title="Flipon group",
)

for i, feature_group in enumerate(feature_groups):
    for j, annotation in enumerate(annotations):
        df = (
            plotly_data_og[
                (plotly_data_og["feature_group"] == feature_group)
                & (plotly_data_og["annotation"] == annotation)
            ]
            .set_index("group")
            .reindex(groups)
            .dropna()
        )

        if not len(df):
            continue

        colors = {
            "G4": "#FE9B54",
            "Z-DNA": "royalblue",
            "SIDD": "#ED4241",
            "H-DNA": "darkorchid",
        }
        # https://stackoverflow.com/questions/70347594/how-to-format-plotly-legend-when-using-marker-color
        for idx, row in df.iterrows():
            sub = pgo.Bar(
                x=[row["count"]],
                y=[idx],
                name=idx,
                textposition="outside" if row["count"] < 2000 else "inside",
                texttemplate="%{x:,d}",
                marker=dict(color=colors[idx]),
                width=1,
                orientation="h",
                showlegend=True if (i == 0 and j == 0) else False,
            )
            fig.add_trace(sub, row=i + 1, col=j + 1)
        fig.update_xaxes(ticks="", row=i + 1, col=j + 1)

fig.update_xaxes(tickformat="s", showgrid=False)
fig.update_xaxes(ticks="outside", row=n_rows)
fig.update_yaxes(type="category", visible=False)

fig.update_annotations(textangle=0, font=dict(size=14), align='left')
fig.update_layout(
    title=f"Flipons ({plotly_data_og['count'].sum():,d})",
    title_x=0.005, title_y=.99,
    height=900,
    width=1600,
    font=dict(family="monospace", size=12),
    uniformtext_minsize=12,
    margin=dict(l=5, t=55, b=55, r=60),
    legend=dict(
        # title_text="Flipon group",
        traceorder="reversed",
        orientation="h",
        yanchor="bottom",
        y=-.07,
        xanchor="right",
        x=0.98,
    ),
    template="ggplot2",
)

fig.write_image(d_img / 'enrichment-table-all-flipons-01-all.png', scale=2)

fig.show()
